<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# SAR Single Node on MovieLens (Python, CPU)

In this example, we will walk through each step of the Simple Algorithm for Recommendation (SAR) algorithm using a Python single-node implementation.

SAR is a fast, scalable, adaptive algorithm for personalized recommendations based on user transaction history. It is powered by understanding the similarity between items, and recommending similar items to those a user has an existing affinity for.

## 1 SAR algorithm

The following figure presents a high-level architecture of SAR. 

At a very high level, two intermediate matrices are created and used to generate a set of recommendation scores:

- An item similarity matrix $S$ estimates item-item relationships.
- An affinity matrix $A$ estimates user-item relationships.

Recommendation scores are then created by computing the matrix multiplication $A\times S$.

Optional steps (e.g. "time decay" and "remove seen items") are described in the details below.

<img src="https://recodatasets.z20.web.core.windows.net/images/sar_schema.svg?sanitize=true">

### 1.1 Compute item co-occurrence and item similarity

SAR defines similarity based on item-to-item co-occurrence data. Co-occurrence is defined as the number of times two items appear together for a given user. We can represent the co-occurrence of all items as a $m\times m$ matrix $C$, where $c_{i,j}$ is the number of times item $i$ occurred with item $j$, and $m$ is the total number of items.

The co-occurence matric $C$ has the following properties:

- It is symmetric, so $c_{i,j} = c_{j,i}$
- It is nonnegative: $c_{i,j} \geq 0$
- The occurrences are at least as large as the co-occurrences. I.e., the largest element for each row (and column) is on the main diagonal: $\forall(i,j) C_{i,i},C_{j,j} \geq C_{i,j}$.

Once we have a co-occurrence matrix, an item similarity matrix $S$ can be obtained by rescaling the co-occurrences according to a given metric. Options for the metric include `Jaccard`, `lift`, and `counts` (meaning no rescaling).


If $c_{ii}$ and $c_{jj}$ are the $i$th and $j$th diagonal elements of $C$, the rescaling options are:

- `Jaccard`: $s_{ij}=\frac{c_{ij}}{(c_{ii}+c_{jj}-c_{ij})}$
- `lift`: $s_{ij}=\frac{c_{ij}}{(c_{ii} \times c_{jj})}$
- `counts`: $s_{ij}=c_{ij}$

In general, using `counts` as a similarity metric favours predictability, meaning that the most popular items will be recommended most of the time. `lift` by contrast favours discoverability/serendipity: an item that is less popular overall but highly favoured by a small subset of users is more likely to be recommended. `Jaccard` is a compromise between the two.


### 1.2 Compute user affinity scores

The affinity matrix in SAR captures the strength of the relationship between each individual user and the items that user has already interacted with. SAR incorporates two factors that can impact users' affinities: 

- It can consider information about the **type** of user-item interaction through differential weighting of different events (e.g. it may weigh events in which a user rated a particular item more heavily than events in which a user viewed the item).
- It can consider information about **when** a user-item event occurred (e.g. it may discount the value of events that take place in the distant past.

Formalizing these factors produces us an expression for user-item affinity:

$$a_{ij}=\sum_k w_k \left(\frac{1}{2}\right)^{\frac{t_0-t_k}{T}} $$

where the affinity $a_{ij}$ for user $i$ and item $j$ is the weighted sum of all $k$ events involving user $i$ and item $j$. $w_k$ represents the weight of a particular event, and the power of 2 term reflects the temporally-discounted event. The $(\frac{1}{2})^n$ scaling factor causes the parameter $T$ to serve as a half-life: events $T$ units before $t_0$ will be given half the weight as those taking place at $t_0$.

Repeating this computation for all $n$ users and $m$ items results in an $n\times m$ matrix $A$. Simplifications of the above expression can be obtained by setting all the weights equal to 1 (effectively ignoring event types), or by setting the half-life parameter $T$ to infinity (ignoring transaction times).

### 1.3 Remove seen item

Optionally we remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

### 1.4 Top-k item calculation

The personalized recommendations for a set of users can then be obtained by multiplying the affinity matrix ($A$) by the similarity matrix ($S$). The result is a recommendation score matrix, where each row corresponds to a user, each column corresponds to an item, and each entry corresponds to a user / item pair. Higher scores correspond to more strongly recommended items.

It is worth noting that the complexity of recommending operation depends on the data size. SAR algorithm itself has $O(n^3)$ complexity. Therefore the single-node implementation is not supposed to handle large dataset in a scalable manner. Whenever one uses the algorithm, it is recommended to run with sufficiently large memory. 

## 2 SAR single-node implementation

The SAR implementation illustrated in this notebook was developed in Python, primarily with Python packages like `numpy`, `pandas`, and `scipy` which are commonly used in most of the data analytics / machine learning tasks. Details of the implementation can be found in [Recommenders/recommenders/models/sar/sar_singlenode.py](../../recommenders/models/sar/sar_singlenode.py).

## 3 SAR single-node based movie recommender

In [22]:
import sys
import logging
import scipy
import numpy as np
import pandas as pd

# from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, mae, rmse, novelty, historical_item_novelty, user_item_serendipity, user_serendipity, serendipity, catalog_coverage, distributional_coverage
from recommenders.models.sar import SAR

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")

System version: 3.9.21 (main, Dec 11 2024, 16:24:11) 
[GCC 11.2.0]
Pandas version: 2.2.3
NumPy version: 1.26.4
SciPy version: 1.13.1


In [23]:
# Top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = "100k"

In [24]:
# set log level to INFO
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

### 3.1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>, <Time>`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. 

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

In [26]:
# data = movielens.load_pandas_df(
#     size=MOVIELENS_DATA_SIZE,
#     header=["userID", "itemID", "rating", "timestamp"],
#     title_col="title",
# )

from recommenders.datasets import dataset_loader

data = dataset_loader.loader("movielens", want_col=["userID", "itemID", "rating", "timestamp"])

# Convert the float precision to 32-bit in order to reduce memory consumption
data["rating"] = data["rating"].astype(np.float32)

data.head()

ImportError: cannot import name 'dataset_loader' from 'recommenders.datasets' (/home/rs/anaconda3/envs/recsys25/lib/python3.9/site-packages/recommenders/datasets/__init__.py)

### 3.2 Split the data using the python random splitter provided in utilities:

We split the full dataset into a `train` and `test` dataset to evaluate performance of the algorithm against a held-out set not seen during training. Because SAR generates recommendations based on user preferences, all users that are in the test set must also exist in the training set. For this case, we can use the provided `python_stratified_split` function which holds out a percentage (in this case 25%) of items from each user, but ensures all users are in both `train` and `test` datasets. Other options are available in the `dataset.python_splitters` module which provide more control over how the split occurs.


In [ ]:
header = {
    "col_user": "userID",
    "col_item": "itemID",
    "col_rating": "rating",
    "col_timestamp": "timestamp",
    "col_prediction": "prediction",
}

In [ ]:
train, test = python_stratified_split(
    data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42
)


In [ ]:
test.head()

,userID,itemID,rating,timestamp,title
56881,1,25,4.0,875071805,"Birdcage, The (1996)"
81463,1,126,2.0,875071713,"Spitfire Grill, The (1996)"
56067,1,45,5.0,875241687,Eat Drink Man Woman (1994)
14425,1,255,2.0,885345822,My Best Friend's Wedding (1997)
35389,1,144,4.0,875073180,Die Hard (1988)


In [ ]:
train.head()

,userID,itemID,rating,timestamp,title
15270,1,54,3.0,878543308,Outbreak (1995)
4411,1,80,4.0,876893008,Hot Shots! Part Deux (1993)
23028,1,138,1.0,878543006,D3: The Mighty Ducks (1996)
3710,1,84,4.0,875072923,Robert A. Heinlein's The Puppet Masters (1994)
37820,1,37,2.0,878543030,Nadja (1994)


In this case, for the illustration purpose, the following parameter values are used:

|Parameter|Value|Description|
|---------|---------|-------------|
|`similarity_type`|`jaccard`|Method used to calculate item similarity.|
|`time_decay_coefficient`|30|Period in days (term of $T$ shown in the formula of Section 1.2)|
|`time_now`|`None`|Time decay reference.|
|`timedecay_formula`|`True`|Whether time decay formula is used.|

In [ ]:
model = SAR(
    similarity_type="jaccard", 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
    **header
)

In [ ]:
model.fit(train)

2025-02-18 22:06:07 INFO     Collecting user affinity matrix
2025-02-18 22:06:07 INFO     Calculating time-decayed affinities
2025-02-18 22:06:07 INFO     Creating index columns
2025-02-18 22:06:07 INFO     Building user affinity sparse matrix
2025-02-18 22:06:07 INFO     Calculating item co-occurrence
2025-02-18 22:06:07 INFO     Calculating item similarity
2025-02-18 22:06:07 INFO     Using jaccard based similarity
2025-02-18 22:06:07 INFO     Done training


In [ ]:
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

2025-02-18 22:06:07 INFO     Calculating recommendation scores
2025-02-18 22:06:07 INFO     Removing seen items


In [ ]:
top_k.head()

,userID,itemID,prediction
0,1,433,2.873124
1,1,204,2.868698
2,1,403,2.868611
3,1,174,2.833488
4,1,70,2.826180


In [ ]:
top = model.recommend_k_items(test, top_k=1, remove_seen=True)

2025-02-18 22:06:07 INFO     Calculating recommendation scores
2025-02-18 22:06:07 INFO     Removing seen items


In [ ]:
top.head()

,userID,itemID,prediction
0,1,433,2.873124
1,2,258,6.591787
2,3,302,5.203198
3,4,258,8.717989
4,5,82,0.769763


The final output from the `recommend_k_items` method generates recommendation scores for each user-item pair, which are shown as follows.

In [ ]:
top_k_with_titles = top_k.join(
    data[["itemID", "title"]].drop_duplicates().set_index("itemID"),
    on="itemID",
    how="inner",
).sort_values(by=["userID", "prediction"], ascending=False)

top_k_with_titles.head(10)

,userID,itemID,prediction,title
9420,943,82,22.481977,Jurassic Park (1993)
9421,943,568,22.248813,Speed (1994)
9422,943,550,22.080580,Die Hard: With a Vengeance (1995)
9423,943,176,21.647137,Aliens (1986)
9424,943,96,21.213240,Terminator 2: Judgment Day (1991)
9425,943,144,20.768581,Die Hard (1988)
9426,943,265,20.762896,"Hunt for Red October, The (1990)"
9427,943,231,20.748540,Batman Returns (1992)
9428,943,423,20.571264,E.T. the Extra-Terrestrial (1982)
9429,943,97,20.566647,Dances with Wolves (1990)


In [ ]:
top_with_titles = top.join(
    data[["itemID", "title"]].drop_duplicates().set_index("itemID"),
    on="itemID",
    how="inner",
).sort_values(by=["userID", "prediction"], ascending=False)

top_with_titles.head(10)

,userID,itemID,prediction,title
942,943,82,22.481977,Jurassic Park (1993)
941,942,69,28.355508,Forrest Gump (1994)
940,941,121,0.117264,Independence Day (ID4) (1996)
939,940,69,7.714822,Forrest Gump (1994)
938,939,117,0.794802,"Rock, The (1996)"
937,938,121,30.209819,Independence Day (ID4) (1996)
936,937,50,0.199970,Star Wars (1977)
935,936,50,8.482319,Star Wars (1977)
934,935,25,2.219059,"Birdcage, The (1996)"
933,934,69,53.636799,Forrest Gump (1994)


### 3.3 Evaluate the results

It should be known that the recommendation scores generated by multiplying the item similarity matrix $S$ and the user affinity matrix $A$ **DOES NOT** have the same scale with the original explicit ratings in the movielens dataset. That is to say, SAR algorithm is meant for the task of *recommending relevent items to users* rather than *predicting explicit ratings for user-item pairs*. 

To this end, ranking metrics like precision@k, recall@k, etc., are more applicable to evaluate SAR algorithm. The following illustrates how to evaluate SAR model by using the evaluation functions provided in Recommenders library.

In [ ]:
# all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_prediction="prediction",
    relevancy_method="top_k",
    k=TOP_K,
)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg_at_k = ndcg_at_k(*args, **kwargs)
eval_precision_at_k = precision_at_k(*args, **kwargs)
eval_recall_at_k = recall_at_k(*args, **kwargs)

In [ ]:
args1 = [test, top]
kwargs1 = dict(
    col_user="userID",
    col_item="itemID",
    col_rating="rating",
    col_prediction="prediction",
    relevancy_method="top_k",
    k=1,
)
eval_ndcg = ndcg_at_k(*args1, **kwargs1)
eval_precision = precision_at_k(*args1, **kwargs1)
eval_recall = recall_at_k(*args1, **kwargs1)

In [ ]:
eval_mae = mae(test, top_k)
eval_rmse = rmse(test, top_k)

In [ ]:
eval_novelty = novelty(train, top)
eval_historical_item_novelty = historical_item_novelty(train, top)
eval_user_item_serendipity = user_item_serendipity(train, top)
eval_user_serendipity = user_serendipity(train, top)
eval_serendipity = serendipity(train, top)
eval_catalog_coverage = catalog_coverage(train, top)
eval_distributional_coverage = distributional_coverage(train, top)

c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\recommenders\evaluation\python_evaluation.py:1435: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avg_novelty = reco_item_novelty.agg({"product": "sum"})[0] / n_recommendations
c:\Users\julia\anaconda3\envs\conda_python3.9\lib\site-packages\recommenders\evaluation\python_evaluation.py:1511: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to p

In [ ]:
print("Precision:\t%f" % eval_precision,
      "Precision@K:\t%f" % eval_precision_at_k,
      "Recall:\t%f" % eval_recall,
      "Recall@K:\t%f" % eval_recall_at_k,
      "MAE:\t%f" % eval_mae,
      "RMSE:\t%f" % eval_rmse,
      "NDCG:\t%f" % eval_ndcg,
      "Novelty:\t%f" % eval_novelty,
      "Serendipity:\t%f" % eval_serendipity,
      "Catalog coverage:\t%f" % eval_catalog_coverage,
      "Distributional coverage:\t%f" % eval_distributional_coverage,
      sep='\n')

Precision:	0.483563
Precision@K:	0.331071
Recall:	0.029189
Recall@K:	0.176837
MAE:	11.394571
RMSE:	24.645236
NDCG:	0.483563
Novelty:	8.211226
Serendipity:	0.671217
Catalog coverage:	0.050817
Distributional coverage:	4.984050


In [ ]:
eval_historical_item_novelty

,itemID,item_novelty
0,1,7.793570
1,2,9.655290
2,3,10.044702
3,4,8.890668
4,5,10.172081
...,...,...
1648,1678,16.194449
1649,1679,16.194449
1650,1680,16.194449
1651,1681,16.194449


In [ ]:
eval_user_item_serendipity

,userID,itemID,user_item_serendipity
0,1,433,0.720375
1,2,258,0.726977
2,3,302,0.751564
3,4,258,0.706664
4,5,82,0.671276
...,...,...,...
938,939,117,0.711782
939,940,69,0.668200
940,941,121,0.610331
941,942,69,0.680842


In [ ]:
eval_user_serendipity

,userID,user_serendipity
0,1,0.720375
1,2,0.726977
2,3,0.751564
3,4,0.706664
4,5,0.671276
...,...,...
938,939,0.711782
939,940,0.668200
940,941,0.610331
941,942,0.680842


## References
Note SAR is a combinational algorithm that implements different industry heuristics. The followings are references that may be helpful in understanding the SAR logic and implementation. 

1. Badrul Sarwar, *et al*, "Item-based collaborative filtering recommendation algorithms", WWW, 2001.
2. Scipy (sparse matrix), url: https://docs.scipy.org/doc/scipy/reference/sparse.html
3. Asela Gunawardana and Guy Shani, "A survey of accuracy evaluation metrics of recommendation tasks", The Journal of Machine Learning Research, vol. 10, pp 2935-2962, 2009.	